In [18]:
import pandas as pd
import glob
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd
import glob
import numpy as np
import re
import csv
import os

from google.colab import drive
drive.mount('/content/drive')

# setting the path for joining multiple files
files = os.path.join("/content/drive/MyDrive/SI650/650 Project/SI650-Data-Recommendation-vertical-search/", "amazon_raw_data*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df = df.dropna(subset = ['URL'])
df.to_csv("amazon_general_queries_data.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# output_file = open("amazon_data.csv","a")
url_data = pd.read_csv("amazon_general_queries_data.csv")
HEADERS = ({'User-Agent': 
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
f = open('scrapped_amazon_general_queries_data.csv', 'w+', encoding="utf-8")
fieldnames = ['productTitle', 'productDescription', 'Rating', 'Price', 'Brand', 'Ingredients', 'About this item',
              'Frequently bought together', 'Products related to this item', 'URL']
writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter=';')

writer.writeheader()

## data cleaning

# list of attrs needed = ['productTitle', 'productDescription', 'Rating', 'Price', 'Brand', 'Ingredients', 'About this item',
#                           'Frequently bought together', 'Products related to this item']
url_data = url_data.dropna(subset=['URL'])
for i in range(len(url_data)):
    data = url_data.iloc[i]
    URL = data.URL
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "lxml")

    # Extracting Title
    try:
        title = soup.find("span",attrs={"id":'productTitle'})
        # print("at 1 title is", title)
        title = title.string.strip().replace(',', '')
        # print("at 1 title is", title)
    #         writer.writerow({'productTitle': title})
    except:
        title = np.nan

    # print("title:", title)

    # Extracting Product Descriptions
    try:
        desc = soup.find("div", attrs={"id":'productDescription'}).find("span").string
    #         writer.writerow({'productDescription': desc})
    except:
        desc = np.nan

    # Rating and Price
    rating = data.Rating

#     try:
#         rating = soup.find("span",attrs={"data-hook":'acr-average-starts-rating-text'})
#     except:
#         rating = np.nan

    price = data.Price

#     try:
#         price = soup.find("span",{"id":'snsDetailPagerPrice', "data-value": True})['data-value'] #.find("span",attrs={"id":'sns-base-price'}).string
#         print("price:",price)
#     except:
#         price = np.nan

    # No. of reviews
    try:
        reviews = soup.find("span",{"id":"acrCustomerReviewText"}).string
    #         writer.writerow({'productDescription': reviews})
    except:
        reviews = np.nan

    # Brand
    try:
        lis = soup.find("table", attrs={"class":"a-normal a-spacing-micro"}).find_all("span")
    #         print("lis ",lis)
        for idx, val in enumerate(lis):
            if(val.string=="Brand"):
                brand = lis[idx+1].string
                break
    #         writer.writerow({'Brand': brand})
        if brand is None:
            brand = np.nan
    except:
        brand = np.nan

    # print("here, brand:",brand)
    # Ingredients
    try:
        flag=0
        lis = soup.find_all("div", attrs={"class" : "a-section content"})
        for idx, val in enumerate(lis):
            st_list = val.find_all(re.compile("h"))
            for idx2, val2 in enumerate(st_list):
                if(val2.string=="Ingredients"):
                    ingredients = lis[idx].text
                    flag=1
                    break
            if(flag):
                break
        if ingredients is None:
            ingredients = np.nan
    #         writer.writerow({'Ingredients': ingredients})
    except:
        ingredients = np.nan


    # About this item
    try:
        lis = soup.find("div", attrs={"id":"feature-bullets"}).find_all("li")
        features = []
        for val in lis:
            features.append(val.string)
    #         writer.writerow({'About this item': features})
    except:
        features = np.nan 

    # Frequently bought together
    # okay

    try:
        fbt = soup.find("div", attrs={"class": "cardRoot bucket"}).find_all("a", href=True)
        fbt_links = []
        for val in fbt:
            fbt_links.append('https://www.amazon.com/'+val['href'])
    #         writer.writerow({'Frequently bought together': fbt_links})
    except:
        fbt_links = np.nan

    # Product related to this item
    try:
        prt = soup.find("div", {"id":"sims-consolidated-2_feature_div"}).find_all("a", href=True)
        dic = {}
        for idx, val in enumerate(prt):
            try:
                temp = val['title']
                dic[temp] = 1
            except:
                temp=1

        prt_list_title = list(dic.keys())
    except:
        prt_list_title = np.nan
        
    writer.writerow({'productTitle': title, 'productDescription': desc, 'Rating': rating, 'Price': price, 'Brand': brand, 'Ingredients': ingredients, 'About this item': features, 'Frequently bought together': fbt_links,'URL': URL})
    

csvreader = csv.reader(x.replace('nan', '') for x in f)
f.close()                            